***GENERATED CODE FOR futurepatient PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import round
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'sex', 'transformation_label': 'String Indexer'}], 'feature': 'sex', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '26', 'mean': '', 'stddev': '', 'min': 'female', 'max': 'male', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'sex'}, {'feature_label': 'sex', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('sex')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'bmi', 'threshold': '25.0', 'transformation_label': 'Binarizer'}], 'feature': 'bmi', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '26', 'mean': '29.27', 'stddev': '6.26', 'min': '18.0', 'max': '47.0', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'bmi'}, {'feature_label': 'bmi', 'threshold': '25.0', 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('bmi')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'smoker', 'transformation_label': 'String Indexer'}], 'feature': 'smoker', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '26', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'smoker'}, {'feature_label': 'smoker', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('smoker')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'region', 'transformation_label': 'String Indexer'}], 'feature': 'region', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '26', 'mean': '', 'stddev': '', 'min': 'northeast', 'max': 'southwest', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'region'}, {'feature_label': 'region', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('region')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
import json
from sklearn.cluster import KMeans
import optuna
import numpy as np
from sklearn import metrics
import pandas as pd


def kmeans(df, TrainingPercent):
    rows = df.count()
    percentrow = TrainingPercent / 100 * rows
    percentrows = int(percentrow)
    df = df.limit(percentrows)
    pd_df = df.toPandas()

    def objective(trial):
        n_clusters = int(trial.suggest_loguniform("n_clusters", 2, 10))
        classifier_obj = KMeans(n_clusters=n_clusters).fit(pd_df)
        labels = classifier_obj.predict(pd_df)
        score = metrics.silhouette_score(pd_df, labels, metric='euclidean')
        return score
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)
    best = study.best_params
    best_param = int(best["n_clusters"])
    model = KMeans(n_clusters=best_param).fit(pd_df)
    labels = model.predict(pd_df)
    silhouette = metrics.silhouette_score(pd_df, labels, metric='euclidean')
    pd_df['prediction'] = pd.DataFrame(labels)
    labels_unique = np.unique(labels)
    totalClusters = len(labels_unique)
    display("totalClusters     : %s" % totalClusters)
    display(pd_df.head())
    return pd_df


class Clustering:

    def run(spark_DF, spark, config):
        stageAttributes = json.loads(config)
        stageAttributes['model']
        trainingPercent = stageAttributes['TrainingPercent']
        autokmeans_obj = kmeans(df=spark_DF, TrainingPercent=trainingPercent)
        [autokmeans_obj]


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run futurepatientHooks.ipynb
try:
	#sourcePreExecutionHook()

	futurepatient = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1717656365785_FuturePatients.csv', 'filename': 'FuturePatients.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'FilePath': '/DataTest/Binarizer/DataFiles/FuturePatients.csv', 'viewFileName': 'FuturePatients.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://pnm.bfirst.ai:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run futurepatientHooks.ipynb
try:
	#transformationPreExecutionHook()

	futurepatientautofe = TransformationMain.run(futurepatient,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "26", "mean": "43.0", "stddev": "14.97", "min": "18", "max": "63", "missing": "0"}, "updatedLabel": "age"}, {"transformationsData": [{"feature_label": "sex", "transformation_label": "String Indexer"}], "feature": "sex", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "26", "mean": "", "stddev": "", "min": "female", "max": "male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "sex"}, {"transformationsData": [{"feature_label": "bmi", "threshold": "25.0", "transformation_label": "Binarizer"}], "feature": "bmi", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "26", "mean": "29.27", "stddev": "6.26", "min": "18.0", "max": "47.0", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "bmi"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "children", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "26", "mean": "1.12", "stddev": "1.37", "min": "0", "max": "5", "missing": "0"}, "updatedLabel": "children"}, {"transformationsData": [{"feature_label": "smoker", "transformation_label": "String Indexer"}], "feature": "smoker", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "26", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "smoker"}, {"transformationsData": [{"feature_label": "region", "transformation_label": "String Indexer"}], "feature": "region", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "26", "mean": "", "stddev": "", "min": "northeast", "max": "southwest", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "region"}, {"transformationsData": [{"feature_label": "charges", "threshold": 13485.14869888476, "transformation_label": "novalue"}], "feature": "charges", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "26", "mean": "13906.69", "stddev": "11201.8", "min": "1909.0", "max": "41676.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "charges"}]}))

	#transformationPostExecutionHook(futurepatientautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run futurepatientHooks.ipynb
try:
	#mlPreExecutionHook()

	futurepatientclustering = Clustering.run(futurepatientautofe,spark,json.dumps( {"autoClustering": 1, "originalfile": "/FileStore/platform/testdata/1717656365785_FuturePatients.csv", "model": "Kmeans", "TrainingPercent": 100, "totalClusters": 2}))

	#mlPostExecutionHook(futurepatientclustering)

except Exception as ex: 
	logging.error(ex)
#spark.stop()
